In [1]:
import torch
import torchaudio
import librosa
import numpy as np
import speech_recognition as sr
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2ForCTC
import miniaudio
import sys
from pathlib import Path
from time import time

/home/scutum/envs/data-science/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
model_path = '/home/scutum/wav2vec2-large-xlsr-turkish-demo/checkpoint-13000'
processor_path = '/home/scutum/wav2vec2-large-xlsr-turkish-demo'

In [3]:
if torch.cuda.is_available():
    model = Wav2Vec2ForCTC.from_pretrained(model_path).to("cuda")
else:
    model = Wav2Vec2ForCTC.from_pretrained(model_path)

In [4]:
processor = Wav2Vec2Processor.from_pretrained(processor_path)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [5]:
r = sr.Recognizer()
r.dynamic_energy_threshold = False
r.energy_threshold = 100
while True:
    with sr.Microphone() as source:
        audio = r.listen(source)
        try:
            start = time()
#           print(f'Google: {r.recognize_google(audio, language="tr-TR").lower()}')
            rd = audio.get_segment()
            rd = rd.get_wav_data()
            da = miniaudio.decode(rd, nchannels=1)
            decoded_audio = torch.FloatTensor(da.samples)
            print(f'decoded audio type: {type(decoded_audio)}')
            sa = librosa.resample(np.asarray(decoded_audio.reshape(-1)), 44_100, 16_000)
            print(f'sa (sample_audio): {sa}, shape {sa.shape}')
            print(f'\nProcessing...')
            with torch.no_grad():
                sa = processor(sa, return_tensors='pt', padding=True)
                print(f'\nPredicting...')
                if torch.cuda.is_available():
                    logits = model(sa.input_values.to("cuda")).logits
                else:
                    logits = model(sa.input_values).logits
                print(f'Prediction')
                pred_ids = torch.argmax(logits, dim=-1)[0]
            end = time()
            print(pred_ids)
            print(f'My Model: {processor.decode(pred_ids)}')
            print(f'time passed: {end - start}')
            torch.cuda.empty_cache()
        except Exception as e:
            print(e)

decoded audio type: <class 'torch.Tensor'>
It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.
sa (sample_audio): [ -5.7529416 -10.926594   -1.822781  ...   3.799       9.60294
   0.       ], shape (547991,)

Processing...

Predicting...
Prediction
tensor([39, 39, 39,  ..., 39, 39, 39], device='cuda:0')
My Model: pevet büyük adan bir kahramandır bir yıldırımdır ama halk kitlesi nekil tabakası nede zamaniyiğını değildir oyıldırımı meydana getiren milletin kendisidir blut kümesi elleklerik oluşturduğunda yıldırım da kendiliğinden oluşur eğer bulutlar elekterikle yüklüde ilkse hiçbir zaman şimşek ve ya yıldırın oluşmaz yalnızca bulut yaninemli bir bu haralinde kalır milletler de böyledir eğer bir millet büyüklük ve kahramanlık özellikleni taşıyorsa ondan yıldırımlar duar kahramanlar çıkar eğer halk kitlesi nemli bir bu har yınından ibaretse hiçbirgüç ondan yıldırım çıkaramaz
time passe

KeyboardInterrupt: 